In [1]:
import orjson
from PIL import Image
import os
from tqdm import tqdm

In [7]:
with open("../../../data/vlm.jsonl", "r") as f:
    raw_labels = [orjson.loads(line) for line in f.readlines()]

In [8]:
ds = []
for sample in tqdm(raw_labels):
    im_fname = sample["image"]
    im_path = os.path.join("../../../data/images", im_fname)
    im = Image.open(im_path)
    for i, anno in enumerate(sample['annotations']):
        x, y, w, h = anno['bbox']
        im_crop = im.crop((x, y, x+w, y+h))
        new_im_path = f"til_siglip_ds/{im_fname.split('.')[0]}_{i}.jpg"
        im_crop.save(new_im_path)
        ds.append({
            "image": new_im_path.split("/")[1],
            "label": anno["caption"]
        })

100%|██████████| 5107/5107 [00:02<00:00, 2372.76it/s]


In [9]:
with open("til_siglip_ds.json", "wb+") as f:
    f.write(orjson.dumps(ds))

In [5]:
with open("XS data/new_annotations.json", "r") as f:
    raw_labels = orjson.loads(f.read())

In [7]:
ds = []
os.makedirs("til_siglip_ds_XS", exist_ok=True)
for sample in tqdm(raw_labels):
    im_fname = sample["image"]
    im_path = os.path.join("XS data/images", im_fname)
    im = Image.open(im_path)
    for i, anno in enumerate(sample['annotations']):
        x, y, w, h = anno['bbox']
        im_crop = im.crop((x, y, x+w, y+h))
        new_im_path = f"til_siglip_ds_XS/{im_fname.split('.')[0]}_{i}.jpg"
        im_crop.save(new_im_path)
        ds.append({
            "image": new_im_path.split("/")[1],
            "label": anno["caption"]
        })

100%|██████████| 2338/2338 [00:26<00:00, 88.85it/s]


In [8]:
with open("til_siglip_ds_XS.json", "wb+") as f:
    f.write(orjson.dumps(ds))

In [11]:
# merge them
with open("til_siglip_ds.json", "r") as f:
    ds1 = orjson.loads(f.read())
with open("til_siglip_ds_XS.json", "r") as f:
    ds2 = orjson.loads(f.read())

ds = ds1 + ds2
with open("til_siglip_ds_merged.json", "wb+") as f:
    f.write(orjson.dumps(ds))